In [23]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import cartopy.crs as ccrs
import cartopy.mpl.ticker as cticker
from cartopy.util import add_cyclic_point

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [53]:
ds_soil = xr.open_dataset('/home/wpratt3/Clim680/Data/soilw.mon.1991-2020.ltm.v2.nc')
ds_climo = ds_soil.groupby('time.month').mean()
ds_anoms = ds_soil.groupby('time.month')-ds_climo

/home/wpratt3/miniforge/envs/climate/lib/python3.12/site-packages/xarray/coding/times.py:992: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/wpratt3/miniforge/envs/climate/lib/python3.12/site-packages/xarray/core/indexing.py:526: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return np.asarray(self.get_duck_array(), dtype=dtype)


In [ ]:
positive_precip = ds_anoms.sel(time=elnino.dropna(dim='time')['time']).mean(dim='time')
negative_precip = ds_anoms.sel(time=lanina.dropna(dim='time')['time']).mean(dim='time')
neutral_precip = ds_anoms.sel(time=neutral.dropna(dim='time')['time']).mean(dim='time')

In [48]:
file_ao_index= '/home/wpratt3/Clim680/Assignment3/ao_index.txt'
np_array = np.loadtxt(fname=file_ao_index)
np_array[:,2]

file_arctic_index = '/home/wpratt3/Clim680/Assignment3/arctic_index.txt'
np_array_index = np.loadtxt(fname=file_arctic_index, skiprows= 1, usecols= (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12), max_rows= 75)
np_array_years = np.loadtxt(fname=file_arctic_index, skiprows= 1, usecols= 0, max_rows= 75)

array([[-6.000e-02,  6.270e-01, -8.000e-03,  5.550e-01,  7.200e-02,
         5.390e-01, -8.020e-01, -8.510e-01,  3.580e-01, -3.790e-01,
        -5.150e-01, -1.928e+00],
       [-8.500e-02, -4.000e-01, -1.934e+00, -7.760e-01, -8.630e-01,
        -9.180e-01,  9.000e-02, -3.770e-01, -8.180e-01, -2.130e-01,
        -6.900e-02,  1.987e+00],
       [ 3.680e-01, -1.747e+00, -1.860e+00,  5.380e-01, -7.730e-01,
        -4.410e-01,  3.830e-01, -3.000e-02, -3.830e-01, -4.370e-01,
        -1.891e+00, -1.827e+00],
       [-1.036e+00, -2.490e-01,  1.068e+00, -1.256e+00, -5.620e-01,
         2.200e-02,  3.330e-01,  8.500e-02,  6.620e-01, -1.950e-01,
         3.540e-01,  5.760e-01],
       [-1.480e-01, -1.810e-01,  4.760e-01,  5.120e-01, -1.656e+00,
        -2.680e-01,  3.410e-01, -1.220e-01,  3.010e-01,  5.130e-01,
        -3.280e-01,  5.530e-01],
       [-1.163e+00, -1.542e+00, -1.568e+00,  1.940e-01,  2.420e-01,
        -2.660e-01,  3.320e-01,  7.600e-01,  3.570e-01,  9.900e-02,
        -1.342e+00,

In [ ]:
#Subset the time of the arctic oscilliation data to match the soils (1991-2020)

In [47]:
#Categorize the arctic oscillation by month
positive = np_array_index.where(np_array_index>=1) # nans where false and unaltered where true if only first parameter given
positive = np_array[:,2].where(np_array[:,2]>=1) # nans where false and unaltered where true if only first parameter given
negative = np_array[:,2].where(np_array[:,2]<=-1)
neutral = np_array[:,2].where((np_array[:,2]>-1) & (np_array[:,2]<1))

AttributeError: 'numpy.ndarray' object has no attribute 'where'